In [1]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [2]:
import os

BASE_DIR = "/content/drive/MyDrive/imar_rag"
RAW_DIR = os.path.join(BASE_DIR, "data/raw")
os.makedirs(RAW_DIR, exist_ok=True)

print("📁 Klasör hazır:", RAW_DIR)


📁 Klasör hazır: /content/drive/MyDrive/imar_rag/data/raw


In [3]:
!pip install pymupdf


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.1/24.1 MB 103.5 MB/s eta 0:00:00


In [4]:
import fitz  # PyMuPDF

pdf_path = "/content/drive/MyDrive/imar_rag/data/raw/planli-20210811131259.pdf"
output_path = "/content/drive/MyDrive/imar_rag/data/planli_alanlar_imar_yonetmeligi.txt"

doc = fitz.open(pdf_path)

all_text = []

for page_num, page in enumerate(doc):
    text = page.get_text()
    all_text.append(f"\n--- SAYFA {page_num + 1} ---\n")
    all_text.append(text)

doc.close()

with open(output_path, "w", encoding="utf-8") as f:
    f.write("\n".join(all_text))

print("✅ PDF metne çevrildi:", output_path)


✅ PDF metne çevrildi: /content/drive/MyDrive/imar_rag/data/planli_alanlar_imar_yonetmeligi.txt


In [5]:
import os
import json

BASE_DIR = "/content/drive/MyDrive/imar_rag/data"

txt_path = os.path.join(BASE_DIR, "planli_alanlar_imar_yonetmeligi.txt")
output_path = os.path.join(BASE_DIR, "chunks.jsonl")

chunk_size = 800
overlap = 150

with open(txt_path, "r", encoding="utf-8") as f:
    text = f.read()

chunks = []
start = 0
chunk_id = 0

while start < len(text):
    end = start + chunk_size
    chunk_text = text[start:end]

    chunks.append({
        "chunk_id": chunk_id,
        "text": chunk_text
    })

    chunk_id += 1
    start = end - overlap

with open(output_path, "w", encoding="utf-8") as f:
    for chunk in chunks:
        f.write(json.dumps(chunk, ensure_ascii=False) + "\n")

print(f"✅ {len(chunks)} adet chunk oluşturuldu")
print("📄 Kaydedildi:", output_path)


✅ 323 adet chunk oluşturuldu
📄 Kaydedildi: /content/drive/MyDrive/imar_rag/data/chunks.jsonl


In [6]:
!pip install chromadb sentence-transformers


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.7/21.7 MB 92.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 278.2/278.2 kB 24.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 87.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 94.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 14.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.0/220.0 kB 22.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.6/71.6 kB 8.5 MB/s eta 0

In [7]:
from chromadb.utils import embedding_functions


In [8]:
import json
import os
import chromadb
from sentence_transformers import SentenceTransformer

BASE_DIR = "/content/drive/MyDrive/imar_rag/data"
chunks_path = f"{BASE_DIR}/chunks.jsonl"
chroma_path = f"{BASE_DIR}/chroma_final"

os.makedirs(chroma_path, exist_ok=True)

# 🔴 Embedding'i BİZ üretiyoruz
model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

documents = []
embeddings = []
ids = []

with open(chunks_path, "r", encoding="utf-8") as f:
    for line in f:
        item = json.loads(line)
        text = item["text"]
        documents.append(text)
        embeddings.append(model.encode(text))
        ids.append(str(item["chunk_id"]))

print("✅ Embedding üretildi:", len(embeddings))


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

✅ Embedding üretildi: 323


In [9]:
client = chromadb.PersistentClient(path=chroma_path)

collection = client.get_or_create_collection(
    name="imar_yonetmeligi_manual"
)

collection.add(
    documents=documents,
    embeddings=embeddings,  # 🔴 KRİTİK
    ids=ids
)

print("✅ Chroma'ya yazıldı")
print("📦 count:", collection.count())


✅ Chroma'ya yazıldı
📦 count: 323


In [10]:
!ls /content/drive/MyDrive/imar_rag/data/chroma_final


c22e4c0f-1a24-4935-92e1-c6c219cb44fa  chroma.sqlite3


In [11]:
collection.count()


323

In [12]:
!pip install groq
import groq

print(groq.__version__)

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.3/138.3 kB 7.8 MB/s eta 0:00:00
1.0.0


In [13]:
from google.colab import userdata
from groq import Groq

groq_api_key = userdata.get('GROQ_API_KEY')
groq_client = Groq(api_key=groq_api_key)

chat_completion = groq_client.chat.completions.create(
    model="llama-3.3-70b-versatile",
    messages=[
        {"role": "user", "content": "Merhaba, çalışıyor musun?"}
    ]
)

print(chat_completion.choices[0].message.content)


Merhaba, evet çalışıyorum. Size nasıl yardımcı olabilirim?


In [14]:
!pip install yfinance
import re
import math
import yfinance as yf
from collections import Counter

In [15]:
# 2. RAG MOTORU (In-Memory Vector Store)
# ------------------------------------------------------------------


class SimpleRAG:
    def __init__(self):
        self.vector_store = []

    def get_embedding(self, text):
        words = re.findall(r'\w+', text.lower())
        return Counter(words)

    def _cosine_similarity(self, vec1, vec2):
        intersection = set(vec1.keys()) & set(vec2.keys())
        numerator = sum([vec1[x] * vec2[x] for x in intersection])
        sum1 = sum([vec1[x]**2 for x in vec1.keys()])
        sum2 = sum([vec2[x]**2 for x in vec2.keys()])
        denominator = math.sqrt(sum1) * math.sqrt(sum2)
        return float(numerator) / denominator if denominator else 0.0

    def ingest(self, text):
        chunks = [s.strip() for s in text.split('.') if s.strip()]
        for chunk in chunks:
            self.vector_store.append({
                "content": chunk,
                "vector": self.get_embedding(chunk)
            })

    def retrieve(self, query, top_k=2):
        query_vec = self.get_embedding(query)
        scores = []
        for item in self.vector_store:
            score = self._cosine_similarity(query_vec, item["vector"])
            scores.append((score, item["content"]))
        scores.sort(key=lambda x: x[0], reverse=True)
        return "\n".join([item[1] for item in scores[:top_k] if item[0] > 0])

# RAG Data Ingestion
rag_engine = SimpleRAG()
mock_data = """
TechCorp employees are allowed to work remotely for 3 days a week.
Lunch is provided for free at the office, and no meal card is given.
TechCorp annual leave entitlement is 14 days for the first year.
Performance bonuses are distributed at the end of each quarter based on EBITDA targets.
"""
rag_engine.ingest(mock_data)


In [16]:
import requests
from bs4 import BeautifulSoup

def web_search_tool(query: str) -> str:
    """
    Use this tool to search the internet for up-to-date or external information.
    Returns a short textual summary from the web.
    """
    print(f"\n🌐 [WEB SEARCH] Searching the web for: '{query}'")

    url = f"https://duckduckgo.com/html/?q={query}"
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})

    if response.status_code != 200:
        return "Web search failed."

    soup = BeautifulSoup(response.text, "html.parser")
    results = soup.find_all("a", class_="result__a")

    snippets = []
    for r in results[:3]:
        snippets.append(r.get_text())

    return " | ".join(snippets) if snippets else "No web results found."


In [17]:
# 3. ARAÇLAR (TOOLS) - DOCSTRINGS EKLENDİ (KRİTİK)
# ------------------------------------------------------------------
import yfinance as yf

def calculator(expression: str) -> str:
    """
    Calculates the result of a mathematical expression.
    e.g. calculator: 100 * 0.15
    """
    try:
        allowed_chars = set("0123456789+-*/(). ")
        if not all(c in allowed_chars for c in expression):
            return "Error: Invalid characters."
        return str(eval(expression))
    except Exception as e:
        return f"Error: {e}"

import chromadb
from sentence_transformers import SentenceTransformer

embedding_model = SentenceTransformer("sentence-transformers/all-MiniLM-L6-v2")

CHROMA_PATH = "/content/drive/MyDrive/imar_rag/data/chroma_final"
client = chromadb.PersistentClient(path=CHROMA_PATH)
collection = client.get_collection(name="imar_yonetmeligi_manual")

def search_knowledge_base(query: str) -> str:
    """
    Use this tool to search the Imar Yonetmeligi knowledge base.
    Returns raw regulation text (context only).
    """
    print(f"\n🔍 [IMAR RAG] Searching for: '{query}'")

    query_vec = embedding_model.encode(query).tolist()

    results = collection.query(
        query_embeddings=[query_vec],
        n_results=3
    )

    contexts = results["documents"][0]

    if not contexts:
        return "No relevant regulation text found."

    return "\n".join(contexts)


def company_address(ticker: str)-> str:
    """
    Returns company address for input ticker.
    e.g. company_address: AAPL
    Returns company address for ticker AAPL which is stock ticker for Apple Inc.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return " ".join([info[key] for key in ["address1", "city", "state", "zip", "country"]])

def fulltime_employees(ticker: str)-> int:
    """
    Returns fulltime employees count for input ticker.
    e.g. fulltime_employees: MSFT
    Returns fulltime employees count for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info["fullTimeEmployees"]


def last_close_price(ticker: str)-> float:
    """
    Returns last close price for input ticker.
    e.g. last_close_price: MSFT
    Returns latest close price for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info["previousClose"]

def EBITDA(ticker: str)-> float:
    """
    Returns EBITDA for input ticker.
    e.g. EBITDA: AAPL
    Returns EBITDA for ticker AAPL which is stock ticker for Apple.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info["ebitda"]

def total_debt(ticker: str)-> float:
    """
    Returns total debt for input ticker.
    e.g. total_debt: AAPL
    Returns total debt for ticker AAPL which is stock ticker for Apple.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info["totalDebt"]

def total_revenue(ticker: str)-> float:
    """
    Returns total revenue for input ticker.
    e.g. total_revenue: MSFT
    Returns total revenue for ticker MSFT which is stock ticker for Microsoft.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info["totalRevenue"]

def debt_to_equity_ratio(ticker: str)-> float:
    """
    Returns debt to equity ratio for input ticker.
    e.g. debt_to_equity_ratio: AAPL
    Returns debt to equity ratio for ticker AAPL which is stock ticker for Apple.
    """
    ticker_obj = yf.Ticker(ticker)
    info = ticker_obj.get_info()

    return info["debtToEquity"]
# Tool List
tools = [calculator, search_knowledge_base, web_search_tool, company_address, fulltime_employees, last_close_price, EBITDA, total_debt, total_revenue]

In [18]:
tools_with_desc = "\n\n".join([tool.__name__ + tool.__doc__ for tool in tools])

print(tools_with_desc)

calculator
    Calculates the result of a mathematical expression.
    e.g. calculator: 100 * 0.15
    

search_knowledge_base
    Use this tool to search the Imar Yonetmeligi knowledge base.
    Returns raw regulation text (context only).
    

web_search_tool
    Use this tool to search the internet for up-to-date or external information.
    Returns a short textual summary from the web.
    

company_address
    Returns company address for input ticker.
    e.g. company_address: AAPL
    Returns company address for ticker AAPL which is stock ticker for Apple Inc.
    

fulltime_employees
    Returns fulltime employees count for input ticker.
    e.g. fulltime_employees: MSFT
    Returns fulltime employees count for ticker MSFT which is stock ticker for Microsoft.
    

last_close_price
    Returns last close price for input ticker.
    e.g. last_close_price: MSFT
    Returns latest close price for ticker MSFT which is stock ticker for Microsoft.
    

EBITDA
    Returns EBITDA for i

In [19]:
known_actions = dict([(tool.__name__, tool) for tool in tools])
known_actions

{'calculator': <function __main__.calculator(expression: str) -> str>,
 'search_knowledge_base': <function __main__.search_knowledge_base(query: str) -> str>,
 'web_search_tool': <function __main__.web_search_tool(query: str) -> str>,
 'company_address': <function __main__.company_address(ticker: str) -> str>,
 'fulltime_employees': <function __main__.fulltime_employees(ticker: str) -> int>,
 'last_close_price': <function __main__.last_close_price(ticker: str) -> float>,
 'EBITDA': <function __main__.EBITDA(ticker: str) -> float>,
 'total_debt': <function __main__.total_debt(ticker: str) -> float>,
 'total_revenue': <function __main__.total_revenue(ticker: str) -> float>}

In [33]:
system_prompt = f"""
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer.

GUIDELINES:
1. Use 'search_knowledge_base' for questions related to Imar Yonetmeligi, zoning rules, construction regulations, emsal, kat sayisi, bina kosullari.
2. Use other tools (yfinance) for public stock market data.
3. Use 'calculator' for ANY math calculation. DO NOT calculate in your head.
4. Use Thought to describe your routing decision.
5. When you have enough information to answer the question, you MUST produce a line starting with "Answer:" and stop.
6. Use 'web_search_tool' ONLY when information is not available in the Imar Yonetmeligi or requires current/external knowledge.
7. If a question mentions a specific year (e.g. 2024), current events, laws, government decisions, or amnesty (imar affı),
you MUST use web_search_tool as the FIRST action.
Do NOT use search_knowledge_base before web_search_tool in such cases.
8.IMPORTANT:
When using web_search_tool, clearly distinguish between:
- enacted laws currently in force
- proposed bills, discussions, or expectations
Do NOT state that an imar affı exists unless it is explicitly enacted and in force.



Your available actions are:

{tools_with_desc}

Example session:

Question: What is half of Apple's last close price?
Thought: I need to get the price first, then divide by 2 using calculator.
Action: last_close_price: AAPL
PAUSE
Observation: 220.0
Thought: Now I need to calculate half of 220.0
Action: calculator: 220.0 / 2
PAUSE
Observation: 110.0
Answer: Half of Apple's last close price is 110.0 USD.
""".strip()
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append({"role": "system", "content": system})

    def __call__(self, message):
        self.messages.append({"role": "user", "content": message})
        result = self.execute()
        self.messages.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        # stop parametresine Llama-3'ün özel token başlangıcını ekliyoruz
        result = groq_client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=self.messages,
        temperature=0,
        stop=["PAUSE", "<|"]
)

        return result.choices[0].message.content

In [21]:
agent = Agent(system_prompt)
agent("Konut alanlarında emsal nasıl hesaplanır?")


"Thought: Emsal hesaplanması için Imar Yönetmeligi'nde belirtilen kurallara bakmak gerekir.\n\nAction: search_knowledge_base: emsal hesaplanması\n\n"

In [22]:
obs = search_knowledge_base("Konut alanlarında emsal hesaplanması")
print(obs)



🔍 [IMAR RAG] Searching for: 'Konut alanlarında emsal hesaplanması'
gili mevzuat, standart ya da bu Yönetmeliğe göre hesap edilen asgari alanları,
c) (Mülga:RG-25/7/2019-30842)
ç) Ortak alan niteliğindeki mescit ve müştemilatın konutlarda 150 m²’si, konut dışı yapılarda 300 m²’si,
d) Bütün cepheleri tamamen gömülü olmak ve ortak alan niteliğinde olmak kaydıyla; otopark alanları ve 22 nci maddede belirtilen tamamen
gömülü ortak alanlar,
e) Ticari amaç içermeyen, ortak alan niteliğindeki çocuk oyun alanlarının ve çocuk bakım ünitelerinin toplam 100 m²’si,
bu hesaba dâhil edilmeksizin emsal haricidir.
(9) Kamu kurum ve kuruluşlarınca yapılacak veya yaptırılacak yapılara; imar planlarında o maksada tahsis edilmiş olmak, plan ve mevzuata
aykırı olmamak üzere mimari, statik, tesisat fenni mesuliyeti ve her türlü sorumlulukların bu kamu kurum ve kuruluşlarınca üs
arası standartlara uygun yapılan bacayı,
şş) Katlar Alanı: Bodrum kat, asma kat, çatı arası piyesi ve açık/kapalı çıkmalar dâhil, k

In [35]:
agent.messages.append({
    "role": "assistant",
    "content": f"Observation: {obs}"
})


In [36]:
final_answer = agent.execute()
print(final_answer)




Thought: Emsal hesaplanması için imar parseli alanının ve yapının inşa edilen tüm kat alanlarının toplamının bilinmesi gerekir. Emsal, katlar alanı katsayısı (KAKS) olarak hesaplanır ve yapının inşa edilen tüm kat alanlarının toplamının imar parseli alanına oranını ifade eder.

Answer: Konut alanlarında emsal, katlar alanı katsayısı (KAKS) olarak hesaplanır ve yapının inşa edilen tüm kat alanlarının toplamının imar parseli alanına oranını ifade eder.


In [37]:
# 5. ORKESTRASYON DÖNGÜSÜ (FINAL)
# ------------------------------------------------------------------
import re

action_re = re.compile(r'^Action: (\w+): (.*)$')

def query(question, max_turns=10):
    i = 0
    react_agent = Agent(system_prompt)
    next_prompt = question
    used_actions = set()   # 🔒 Loop prevention

    print(f"\n{'='*40}\nQUESTION: {question}\n{'='*40}")

    while i < max_turns:
        i += 1

        result = react_agent(next_prompt)
        print(f"\033[94m{result}\033[0m")

        actions = [action_re.match(a) for a in result.split('\n') if action_re.match(a)]

        if actions:
            action, action_input = actions[0].groups()

            if action not in known_actions:
                raise Exception(f"Unknown action: {action}: {action_input}")

            # 🔁 Aynı tool tekrar çağrılmasın
            if action in used_actions:
                print(f"=> {action} already used, skipping to prevent loop")
                next_prompt = (
                    "Observation: This tool was already used and did not provide new information. "
                    "Use existing observations to answer the question."
                )
                continue

            used_actions.add(action)

            print(f"=> Running {action} with input: {action_input}")

            try:
                observation = known_actions[action](action_input)
            except Exception as e:
                observation = f"Error executing tool: {e}"

            print(f"Observation: {observation}")
            next_prompt = f"Observation: {observation}"

        else:
            if "Answer:" in result:
                print("\n\033[92m--- FINAL ANSWER FOUND ---\033[0m")
                print(result.split("Answer:")[-1].strip())
            return


In [26]:
query("Konut alanlarında emsal nasıl hesaplanır?")



QUESTION: Konut alanlarında emsal nasıl hesaplanır?
Thought: Emsal hesaplanması için Imar Yönetmeligi'nde belirtilen kurallara bakmak gerekir.

Action: search_knowledge_base: emsal hesaplanması


=> Running search_knowledge_base with input: emsal hesaplanması

🔍 [IMAR RAG] Searching for: 'emsal hesaplanması'
Observation: a projeleri,
9) Otopark, sığınak ve ağaç hesaplarından,
oluşur.
b) Ayrıca ilgili mühendis ve mimarlarca hazırlanan; asansör avan projesi, ısı ve su yalıtım projesi veya raporu, gürültüye karşı korunma proje
veya raporu, yerleşme ve yapının özelliğine göre ilgili idarece istenecek peyzaj projesi de eklenir.
(3) Parsel alanının, parseldeki her bir binanın emsale konu alan büyüklüğünün, parseldeki tüm binaların toplam emsale konu alan büyüklüğünün,
yapı inşaat alanının, toplam yapı inşaat alanının, binanın ve binaların taban alanının ve taban alanı katsayısının, kat alanı katsayısının (emsal), parsel
üzerindeki yapıların blok numaralarının, bloklardaki bağımsız bölüm num

In [27]:
query("1000 m² arsada emsal 1.50 ise maksimum inşaat alanı kaç m²’dir?")



QUESTION: 1000 m² arsada emsal 1.50 ise maksimum inşaat alanı kaç m²’dir?
Thought: To find the maximum construction area, we need to multiply the plot area by the emsal value.

Action: calculator: 1000 * 1.50


=> Running calculator with input: 1000 * 1.50
Observation: 1500.0
Thought: The result of the calculation is the maximum construction area.

Answer: Maksimum inşaat alanı 1500 m²'dir.

--- FINAL ANSWER FOUND ---
Maksimum inşaat alanı 1500 m²'dir.


In [38]:
query("2024 yılında Türkiye’de imar affı var mı ve emsal hesabını etkiler mi?")



QUESTION: 2024 yılında Türkiye’de imar affı var mı ve emsal hesabını etkiler mi?
Thought: 2024 yılında Türkiye'de imar affı olup olmadığını ve emsal hesabını etkileyip etkilemediğini öğrenmek için web_search_tool'u kullanacağım.

Action: web_search_tool: 2024 Türkiye imar affı emsal hesabı


=> Running web_search_tool with input: 2024 Türkiye imar affı emsal hesabı

🌐 [WEB SEARCH] Searching the web for: '2024 Türkiye imar affı emsal hesabı'
Observation: imar Barışı - İmar Affı (2025 Güncel) - Aslan Duran Hukuk Bürosu | İMAR AFFI VE ŞARTLARI (İMAR BARIŞI) - 2024 - Harbiye Hukuk | imar Barışı - İmar Affı Şartları ve Başvurusu (2026 Güncel)
Thought: Gözlemlerime göre, 2024 yılında Türkiye'de imar affı ile ilgili bilgi bulunmakta, ancak bu bilgilerin güncelliği ve doğruluğu teyit edilmelidir. İmar affının emsal hesabını etkileyip etkilemediğini öğrenmek için daha fazla bilgiye ihtiyacım var.

Action: web_search_tool: 2024 Türkiye imar affı emsal hesabı etkileri


=> web_search_tool alread

In [39]:
query("2024’te Türkiye’de imar affı ile ilgili güncel bir gelişme var mı? Varsa (ya da yoksa) bunun emsal (KAKS) hesabını değiştirip değiştirmediğini açıklayarak söyle. Ardından imar yönetmeliğine göre emsal tanımını knowledge base’den al ve KAKS=1.50, parsel=1200 m² için maksimum inşaat alanını calculator ile hesaplayıp sonucu m² olarak ver.")



QUESTION: 2024’te Türkiye’de imar affı ile ilgili güncel bir gelişme var mı? Varsa (ya da yoksa) bunun emsal (KAKS) hesabını değiştirip değiştirmediğini açıklayarak söyle. Ardından imar yönetmeliğine göre emsal tanımını knowledge base’den al ve KAKS=1.50, parsel=1200 m² için maksimum inşaat alanını calculator ile hesaplayıp sonucu m² olarak ver.
Thought: 2024 yılında Türkiye'de imar affı ile ilgili güncel bir gelişme olup olmadığını öğrenmek için web_search_tool'u kullanacağım.

Action: web_search_tool: 2024 Türkiye imar affı


=> Running web_search_tool with input: 2024 Türkiye imar affı

🌐 [WEB SEARCH] Searching the web for: '2024 Türkiye imar affı'
Observation: İMAR AFFI VE ŞARTLARI (İMAR BARIŞI) - 2024 - Harbiye Hukuk | imar Barışı - İmar Affı (2025 Güncel) - Aslan Duran Hukuk Bürosu | İmar Affı (İmar Barışı) Nedir? İmar Affı Şartları ve Başvuru 2024
Thought: Gözlemlerime göre, 2024 yılında Türkiye'de imar affı ile ilgili güncel gelişmeler mevcut. Ancak, emsal (KAKS) hesabını deği

In [40]:
query("İmar affı kapsamına giren yapıların, sonradan yapılacak yeni inşaatlarda emsal (KAKS) hesabına etkisi olur mu?")


QUESTION: İmar affı kapsamına giren yapıların, sonradan yapılacak yeni inşaatlarda emsal (KAKS) hesabına etkisi olur mu?
Thought: İmar affı ve emsal hesabına ilişkin bilgi almak için web_search_tool'u kullanacağım.

Action: web_search_tool: İmar affı emsal hesabına etkisi


=> Running web_search_tool with input: İmar affı emsal hesabına etkisi

🌐 [WEB SEARCH] Searching the web for: 'İmar affı emsal hesabına etkisi'
Observation: İmar Affı Nedir? Başvuru Süreci ve Önemi - arixinvest.com | İMAR AFFI VE ŞARTLARI (İMAR BARIŞI) - 2024 - Harbiye Hukuk | imar Barışı - İmar Affı (2025 Güncel) - Aslan Duran Hukuk Bürosu
Thought: İmar affı ve emsal hesabına ilişkin bilgi almak için search_knowledge_base'i kullanacağım, çünkü observation'da yeterli bilgi bulunmuyor.

Action: search_knowledge_base: İmar affı emsal hesabına etkisi


=> Running search_knowledge_base with input: İmar affı emsal hesabına etkisi

🔍 [IMAR RAG] Searching for: 'İmar affı emsal hesabına etkisi'
Observation: a projeleri,
9)

In [42]:
query("""
İmar yönetmeliğine göre konut alanlarında emsal (KAKS) hesabı nasıl tanımlanır?

İstanbul için güncel dolar/TL kurunu web üzerinden öğren.
Ardından 800 m² büyüklüğünde bir parsel için,
emsal (KAKS) değeri 1.25 olan bir imar parselinde
maksimum inşaat alanını hesapla.

Son olarak, bulunan maksimum inşaat alanının
1 m² inşaat maliyeti 900 USD kabul edilirse,
toplam inşaat maliyetini TL cinsinden hesapla.

""")



QUESTION: 
İmar yönetmeliğine göre konut alanlarında emsal (KAKS) hesabı nasıl tanımlanır?

İstanbul için güncel dolar/TL kurunu web üzerinden öğren.
Ardından 800 m² büyüklüğünde bir parsel için,
emsal (KAKS) değeri 1.25 olan bir imar parselinde
maksimum inşaat alanını hesapla.

Son olarak, bulunan maksimum inşaat alanının
1 m² inşaat maliyeti 900 USD kabul edilirse,
toplam inşaat maliyetini TL cinsinden hesapla.


Thought: İlk olarak, İmar Yönetmeliğine göre konut alanlarında emsal (KAKS) hesabı nasıl tanımlanır, bunu search_knowledge_base kullanarak öğreneceğim.

Action: search_knowledge_base: İmar Yönetmeliğine göre konut alanlarında emsal (KAKS) hesabı


=> Running search_knowledge_base with input: İmar Yönetmeliğine göre konut alanlarında emsal (KAKS) hesabı

🔍 [IMAR RAG] Searching for: 'İmar Yönetmeliğine göre konut alanlarında emsal (KAKS) hesabı'
Observation: rme yapılabilir.
(3) Arka bahçelerde 2.00 metreden fazla olmamak ve (±0.00) kotunu geçmemek koşuluyla dolgu yapılabilir